# Starts here

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel(r'C:\Users\guilherme.oliveira\Desktop\programa_massa\archives\MonitorFlexExportacao.xlsx', skiprows=6)
Bdf = pd.read_excel(r'C:\Users\guilherme.oliveira\Desktop\programa_massa\archives\BASQUETE.xlsx')
Cdf = pd.read_excel(r'C:\Users\guilherme.oliveira\Desktop\programa_massa\archives\COBERTURA.xlsx')

df = df.iloc[:-1]

print(df)

# Data Cleansing

In [ ]:
df.drop(df[df['Categoria'] == 'CAMPANHAS BENEFICENTES SOCIAIS'].index, inplace = True)
df.drop(df[df['Categoria'] == 'CAMPANHAS PARTIDARIAS'].index, inplace = True)

print(df)

In [ ]:
conditions = (
    df['Anunciante'].str.contains('GLOBO') |
    df['Anunciante'].str.contains('BANDEIRANTES') |
    df['Anunciante'].str.contains('RECORD') |
    df['Anunciante'].str.contains('GAZETA') |
    df['Anunciante'].str.contains('CNT') |
    df['Anunciante'].str.contains('JOVEM PAN') |
    df['Anunciante'].str.contains('RADIO') |
    df['Anunciante'].str.contains('MASSA') |
    df['Anunciante'].str.contains('TELEVISAO') |
    df['Anunciante'].str.contains('TV') |
    df['Anunciante'].str.contains('SBT') |
    df['Anunciante'].str.contains('RIC') |
    (df['Marca'].str.contains('CARTOLA', na=False)) |
    df['Anunciante'].str.contains('{DESCONHECIDO}') |
    df['Anunciante'].str.contains('TSE')
)

df.drop(df[conditions].index, inplace=True)

print(df)

# Correcting Faulty Data

In [ ]:
#function to extract a value from the row (finding the city name from the prefecture name)
def extract_value(row):
    start = 'PREF MUN '
    end = ' (GMP)'
    value = row['Anunciante']
    if start in value and end in value:
        extracted_value = value.split(start)[1].split(end)[0]
        return extracted_value
    else:
        return row['Cidade Autorização']

#apply the city name from the correct prefecture to the city column
df['Cidade Autorização'] = df.apply(extract_value, axis=1)

#rename the column from the gov est pr to match it's location
df['UF Autorização'] = np.where(df['Anunciante'].str.contains('GOV'), 'PARANA', df['UF Autorização'])
df['Cidade Autorização'] = np.where(df['Anunciante'].str.contains('GOV'), 'CURITIBA', df['Cidade Autorização'])

print(df)

In [ ]:
clients_maringa = (
    (df['Marca'] == 'CANCAO') |
    df['Marca'].str.contains('AMIGAO', na=False) |
    df['Marca'].str.contains('ZAELI', na=False) |
    ((df['Marca'].str.contains('IMPACTO', na=False)) & (df['Praça'].str.contains('MARINGA', na=False))) |
    ((df['Marca'].str.contains('SOLUCIONADOR', na=False)) & (df['Praça'].str.contains('MARINGA', na=False))) |
    df['Marca'].str.contains('COAMO', na=False) |
    df['Marca'].str.contains('UNICESUMAR', na=False)
)
clients_curitiba = (
    df['Marca'].str.contains('PEDROSO', na=False) |
    (df['Anunciante'] == 'ALTHAIA')
)
clients_cascavel = (
    (df['Marca'].str.contains('IMPACTO PRIME', na=False)) & (df['Praça'].str.contains('CASCAVEL', na=False))
)
clients_londrina = (
    df['Marca'].str.contains('SUPER MUFFATO', na=False)
)

# Rename the city column that commonly mismatches its real location
df.loc[clients_maringa, ['Cidade Autorização', 'UF Autorização']] = ['MARINGA', 'PARANA']
df.loc[clients_curitiba, ['Cidade Autorização', 'UF Autorização']] = ['CURITIBA', 'PARANA']
df.loc[clients_londrina, ['Cidade Autorização', 'UF Autorização']] = ['LONDRINA', 'PARANA']
df.loc[clients_cascavel, ['Cidade Autorização', 'UF Autorização']] = ['CASCAVEL', 'PARANA']

# Print the corrected DataFrame
print(df)

# New columns and filling those columns

In [ ]:
#creates a few columns
df['Vl Tab (000)'] = df['INV(000)'] * 1000
df['Desconto'] = 0
df['Valor Líquido Projetado'] = 0
df['Cobertura'] = None
df['Região'] = None
df['Mercado'] = None
df['Data'] = '01/07/2023'

df.reset_index(inplace=True, drop=True)

print(df.columns)

In [ ]:
#function to determine coverage:
def determine_coverage(row, Cdf):
    cidade_autorizacao = row['Cidade Autorização'].upper()
    
    if 'COBERTURA' in Cdf.columns and pd.notna(Cdf.loc[Cdf['Municipio'] == cidade_autorizacao, 'COBERTURA']).any():
        return Cdf.loc[Cdf['Municipio'] == cidade_autorizacao, 'COBERTURA'].values[0]
    else:
        return 'IMPORT'
    
#function to determine our coverage region:
cidade_region_map = Cdf.set_index('Municipio')['Região'].to_dict()

#function to determine our market:
def set_market(row):
    UF = row['UF Autorização']
    anunciante = row['Anunciante'].upper()
    cidade_autorizacao = row['Cidade Autorização'].upper()

    if 'PREF' in anunciante and (cidade_autorizacao in ['CURITIBA', 'MARINGA', 'CASCAVEL', 'TOLEDO', 'FOZ DO IGUACU', 'LONDRINA']):
        return 'PREF SEDE'
    elif 'PREF' in anunciante:
        return 'PREF'
    elif 'GOV' in anunciante:
        return 'GOVERNO'
    elif 'ASSEMBLEIA' in anunciante:
        return 'ASSEMBLEIA'
    
    return 'LOCAL' if UF == 'PARANA' else 'IMPORT'

#fill some of our Columns:
df['Cobertura'] = df.apply(determine_coverage, args=(Cdf,), axis=1)
df['Região'] = df['Cidade Autorização'].map(cidade_region_map).fillna('IMPORT')
df['Mercado'] = df.apply(set_market, axis=1)

#print our updated DataFrame
print(df)

#### Setting our discounts

In [ ]:
def discount_giver(row):
    emissora = row['Emissora TV']
    coverage = row['Cobertura']
    praca = row['Praça']
    anunciante = row['Anunciante']
    
    #SBT discounts
    if 'SBT' in emissora and 'MERCHANDISING' not in praca:
        return 0.94
    elif 'SBT' in emissora and 'MERCHANDISING' in praca:
        return 0.9
    #BAND discounts
    elif 'BANDEIRANTES' in emissora and 'MERCHANDISING' not in praca and (praca in ['LONDRINA', 'MARINGA', 'CURITIBA']):
        return 0.96
    elif 'BANDEIRANTES' in emissora and 'MERCHANDISING' in praca and (praca in ['LONDRINA', 'MARINGA', 'CURITIBA']):
        return 0.955
    elif 'BANDEIRANTES' in emissora and 'MERCHANDISING' not in praca and 'CASCAVEL' in praca:
        return 0.92
    elif 'BANDEIRANTES' in emissora and 'MERCHANDISING' in praca and 'CASCAVEL' in praca:
        return 0.89
    #CNT discounts
    elif 'CNT' in emissora and 'CURITIBA' in praca:
        return 0.9
    #GLOBO discounts
    elif 'GLOBO' in emissora and 'MERCHANDISING' not in praca and (praca in ['MARINGA', 'LONDRINA', 'FOZ']):
        return 0.3
    elif 'GLOBO' in emissora and 'MERCHANDISING' in praca and (praca in ['MARINGA', 'LONDRINA', 'FOZ']):
        return 0.29
    elif 'GLOBO' in emissora and 'MERCHANDISING' not in praca and (praca in ['PARANAVAI', 'PONTA GROSSA', 'GUARAPUAVA']):
        return 0.4
    elif 'GLOBO' in emissora and 'MERCHANDISING' not in praca and (praca in ['CURITIBA', 'CASCAVEL']):
        return 0.25
    elif 'GLOBO' in emissora and 'MERCHANDISING' not in praca and (praca in ['CURITIBA', 'CASCAVEL']):
        return 0.24
    
    #Import discounts
    elif 'BANDEIRANTES' in emissora and 'IMPORT' in coverage:
        return 0.95
    elif 'SBT' in emissora and 'IMPORT' in coverage:
        return 0.9
    elif 'GLOBO' in emissora and 'IMPORT' in coverage:
        return 0.1
    elif 'CNT' in emissora and 'IMPORT' in coverage:
        return 0.96
    elif 'RECORD' in emissora and 'IMPORT' in coverage:
        return 0.82
    
    #Specific Discounts
    #SBT discounts:
    elif 'SBT' in emissora and 'CONDOR' in anunciante and 'MERCHANDISING' not in praca:
        return 0.89
    elif 'SBT' in emissora and 'CONDOR' in anunciante and 'MERCHANDISING' in praca:
        return 0.88
    elif 'SBT' in emissora and 'SUPER MUFFATO' in anunciante and 'MERCHANDISING' not in praca:
        return 0.89
    elif 'SBT' in emissora and 'SUPER MUFFATO' in anunciante and 'MERCHANDISING' in praca:
        return 0.88
    elif 'SBT' in emissora and 'ALIANCA' in anunciante and 'MERCHANDISING' not in praca:
        return 0.9950
    elif 'SBT' in emissora and 'ALIANCA' in anunciante and 'MERCHANDISING' in praca:
        return 0.93
    elif 'SBT' in emissora and 'RIO VERDE' in anunciante:
        return 0.98
    elif 'SBT' in emissora and 'ODONTO EXCELLENCE' in anunciante:
        return 0.96
    #BAND discounts
    elif 'BANDEIRANTES' in emissora and 'MALUCELLLI' in anunciante:
        return 0.98
    elif 'BANDEIRANTES' in emissora and 'PONTO DE VISAO' in anunciante:
        return 0.9850
    elif 'BANDEIRANTES' in emissora and 'O SOLUCIONADOR' in anunciante:
        return 0.98
    elif 'BANDEIRANTES' in emissora and 'SUPER MUFFATO' in anunciante:
        return 0.97
    #GLOBO discounts
    elif 'GLOBO' in emissora and 'COORITIBA FOOT BALL CLUB' in anunciante:
        return 0.5
    elif 'GLOBO' in emissora and 'PONTO DE VISÃO' in anunciante:
        return 0.7
    elif 'GLOBO' in emissora and 'KURTEN' in anunciante:
        return 0.55
    else:
        return 0

    
df['Desconto'] = df.apply(discount_giver, axis=1)

print(df)

#### Applying those discounts

In [ ]:
# Apply some discounts
mask1 = (df['Desconto'] != 0) & (df['Agência'] == '{DIRETO}')  # Check for direct agency
mask2 = (df['Desconto'] != 0) & (df['Agência'] != '{DIRETO}')  # Check for non-direct agency

df['Valor Líquido Projetado'] = 0  # Initialize the column with zeros

df.loc[mask1, 'Valor Líquido Projetado'] = df['Vl Tab (000)'] * (1 - df['Desconto'])
df.loc[mask2, 'Valor Líquido Projetado'] = df['Vl Tab (000)'] * (1 - df['Desconto']) * (1 - 0.2)

print(df)

# Applying Prices from the Bdf Data Frame

#### Filtering Bdf

In [ ]:
#here the fun begins:
condition1 = Bdf['Mês'] == '2023-07-01'
condition2 = Bdf['Exibição'].isin(['CURITIBA', 'LONDRINA', 'TOLEDO', 'MARINGÁ'])
condition3 = Bdf['Contato'] != 'PERMUTA'

Bdf = Bdf[condition1 & condition2 & condition3]

print(Bdf)

In [ ]:
#first and foremost, let's find out how much we sold to our big guys at the gov and assem.
condition1 = Bdf['Contato'].isin(['GOVERNO', 'ASSEMBLEIA'])

#now that we filtered our big guys, let's start with Curitiba
condition2 = Bdf['Exibição'] == 'CURITIBA'

#now, that we've filtered everything, we'll start making some stuff
Bdf_CWB = Bdf[condition1 & condition2]

#now, we group some data:
Bdf_CWB_group = Bdf_CWB.groupby(['Contato']).agg({'Valor Líquido': 'sum'}).reset_index()

#print this shit out
print(Bdf_CWB_group)

#### Setting Gov and Assem prices for Curitiba

In [ ]:
#this function will search through our filters and will value the rows that meet the parameters
def set_govprices(row, Bdf_CWB):
    anunciante = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    
    if 'GOV' in anunciante and 'CURITIBA' in praca and 'RECORD' in emissora:
        filtered_rows = Bdf_CWB[Bdf_CWB['Contato'] == 'GOVERNO']
        
        sum_value = filtered_rows['Valor Líquido'].sum()
    
        # Debug print statements
        print(f"Row Index: {row.name}")
        print(f"Filtered Rows:\n{filtered_rows}")
    
        # Create a mask for the filtered rows
        mask = (df['Anunciante'].str.upper().str.contains('GOV')) & (df['Praça'].str.upper().str.contains('CURITIBA')) & (df['Emissora TV'].str.upper().str.contains('RECORD'))
        
        # Count the number of rows that meet the filter criteria in your main DataFrame (df)
        num_filtered_rows_in_df = len(df[mask])
        
        if num_filtered_rows_in_df > 0:
            # Distribute the sum_value equally among the filtered rows in your main DataFrame
            value_per_row = sum_value / num_filtered_rows_in_df
            
            # Update the 'Valor Líquido Projetado' column for the filtered rows
            df.loc[mask, 'Valor Líquido Projetado'] = value_per_row

# Apply the function to each row
df.apply(set_govprices, args=(Bdf_CWB,), axis=1)

def set_assemprices(row, Bdf_CWB):
    anunciante = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    
    if 'ASSEMBLEIA' in anunciante and 'CURITIBA' in praca and 'RECORD' in emissora:
        filtered_rows = Bdf_CWB[Bdf_CWB['Contato'] == 'ASSEMBLEIA']
        
        sum_value = filtered_rows['Valor Líquido'].sum()
    
        # Debug print statements
        print(f"Row Index: {row.name}")
        print(f"Filtered Rows:\n{filtered_rows}")
    
        # Create a mask for the filtered rows
        mask = (df['Anunciante'].str.upper().str.contains('ASSEMBLEIA')) & (df['Praça'].str.upper().str.contains('CURITIBA')) & (df['Emissora TV'].str.upper().str.contains('RECORD'))
        
        # Count the number of rows that meet the filter criteria in your main DataFrame (df)
        num_filtered_rows_in_df = len(df[mask])
        
        if num_filtered_rows_in_df > 0:
            # Distribute the sum_value equally among the filtered rows in your main DataFrame
            value_per_row = sum_value / num_filtered_rows_in_df
            
            # Update the 'Valor Líquido Projetado' column for the filtered rows
            df.loc[mask, 'Valor Líquido Projetado'] = value_per_row

# Apply the function to each row

df.apply(set_assemprices, args=(Bdf_CWB,), axis=1)

# Print the modified DataFrame
print(df)

#### Setting Gov and Assem prices for Maringa

In [ ]:
#this function will search through our filters and will value the rows that meet the parameters
def set_govprices(row, Bdf_MAR):
    anunciante = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    
    if 'GOV' in anunciante and 'MARINGA' in praca and 'RECORD' in emissora:
        filtered_rows = Bdf[(Bdf['Contato'] == 'GOVERNO') & (Bdf['Exibição'].str.contains('MARINGÁ'))]
        
        sum_value = filtered_rows['Valor Líquido'].sum()
    
        # Debug print statements
        print(f"Row Index: {row.name}")
        print(f"Filtered Rows:\n{filtered_rows}")
    
        # Create a mask for the filtered rows
        mask = (df['Anunciante'].str.upper().str.contains('GOV')) & (df['Praça'].str.upper().str.contains('MARINGA')) & (df['Emissora TV'].str.upper().str.contains('RECORD'))
        
        # Count the number of rows that meet the filter criteria in your main DataFrame (df)
        num_filtered_rows_in_df = len(df[mask])
        
        if num_filtered_rows_in_df > 0:
            # Distribute the sum_value equally among the filtered rows in your main DataFrame
            value_per_row = sum_value / num_filtered_rows_in_df
            
            # Update the 'Valor Líquido Projetado' column for the filtered rows
            df.loc[mask, 'Valor Líquido Projetado'] = value_per_row

# Apply the function to each row
df.apply(set_govprices, args=(Bdf,), axis=1)

def set_assemprices(row, Bdf):
    anunciante = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    
    if 'ASSEMBLEIA' in anunciante and 'MARINGA' in praca and 'RECORD' in emissora:
        filtered_rows = Bdf[(Bdf['Contato'] == 'ASSEMBLEIA') & (Bdf['Exibição'].str.contains('MARINGÁ'))]
        
        sum_value = filtered_rows['Valor Líquido'].sum()
    
        # Debug print statements
        print(f"Row Index: {row.name}")
        print(f"Filtered Rows:\n{filtered_rows}")
    
        # Create a mask for the filtered rows
        mask = (df['Anunciante'].str.upper().str.contains('ASSEMBLEIA')) & (df['Praça'].str.upper().str.contains('MARINGA')) & (df['Emissora TV'].str.upper().str.contains('RECORD'))
        
        # Count the number of rows that meet the filter criteria in your main DataFrame (df)
        num_filtered_rows_in_df = len(df[mask])
        
        if num_filtered_rows_in_df > 0:
            # Distribute the sum_value equally among the filtered rows in your main DataFrame
            value_per_row = sum_value / num_filtered_rows_in_df
            
            # Update the 'Valor Líquido Projetado' column for the filtered rows
            df.loc[mask, 'Valor Líquido Projetado'] = value_per_row

# Apply the function to each row

df.apply(set_assemprices, args=(Bdf,), axis=1)

# Print the modified DataFrame
print(df)

#### Setting Gov and Assem prices for Londrina

In [ ]:
#this function will search through our filters and will value the rows that meet the parameters
def set_govprices(row, Bdf):
    anunciante = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    
    if 'GOV' in anunciante and 'LONDRINA' in praca and 'RECORD' in emissora:
        filtered_rows = Bdf[(Bdf['Contato'] == 'GOVERNO') & (Bdf['Exibição'].str.contains('LONDRINA'))]
        
        sum_value = filtered_rows['Valor Líquido'].sum()
    
        # Debug print statements
        print(f"Row Index: {row.name}")
        print(f"Filtered Rows:\n{filtered_rows}")
    
        # Create a mask for the filtered rows
        mask = (df['Anunciante'].str.upper().str.contains('GOV')) & (df['Praça'].str.upper().str.contains('MARINGA')) & (df['Emissora TV'].str.upper().str.contains('RECORD'))
        
        # Count the number of rows that meet the filter criteria in your main DataFrame (df)
        num_filtered_rows_in_df = len(df[mask])
        
        if num_filtered_rows_in_df > 0:
            # Distribute the sum_value equally among the filtered rows in your main DataFrame
            value_per_row = sum_value / num_filtered_rows_in_df
            
            # Update the 'Valor Líquido Projetado' column for the filtered rows
            df.loc[mask, 'Valor Líquido Projetado'] = value_per_row

# Apply the function to each row
df.apply(set_govprices, args=(Bdf,), axis=1)

def set_assemprices(row, Bdf):
    anunciante = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    
    if 'ASSEMBLEIA' in anunciante and 'LONDRINA' in praca and 'RECORD' in emissora:
        filtered_rows = Bdf[(Bdf['Contato'] == 'LONDRINA') & (Bdf['Exibição'].str.contains('MARINGÁ'))]
        
        sum_value = filtered_rows['Valor Líquido'].sum()
    
        # Debug print statements
        print(f"Row Index: {row.name}")
        print(f"Filtered Rows:\n{filtered_rows}")
    
        # Create a mask for the filtered rows
        mask = (df['Anunciante'].str.upper().str.contains('ASSEMBLEIA')) & (df['Praça'].str.upper().str.contains('MARINGA')) & (df['Emissora TV'].str.upper().str.contains('RECORD'))
        
        # Count the number of rows that meet the filter criteria in your main DataFrame (df)
        num_filtered_rows_in_df = len(df[mask])
        
        if num_filtered_rows_in_df > 0:
            # Distribute the sum_value equally among the filtered rows in your main DataFrame
            value_per_row = sum_value / num_filtered_rows_in_df
            
            # Update the 'Valor Líquido Projetado' column for the filtered rows
            df.loc[mask, 'Valor Líquido Projetado'] = value_per_row

# Apply the function to each row

df.apply(set_assemprices, args=(Bdf,), axis=1)

# Print the modified DataFrame
print(df)

#### Setting Gov and Assem prices for Cascavel

In [ ]:
#this function will search through our filters and will value the rows that meet the parameters
def set_govprices(row, Bdf):
    anunciante = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    
    if 'GOV' in anunciante and 'LONDRINA' in praca and 'RECORD' in emissora:
        filtered_rows = Bdf[(Bdf['Contato'] == 'GOVERNO') & (Bdf['Exibição'].str.contains('LONDRINA'))]
        
        sum_value = filtered_rows['Valor Líquido'].sum()
    
        # Debug print statements
        print(f"Row Index: {row.name}")
        print(f"Filtered Rows:\n{filtered_rows}")
    
        # Create a mask for the filtered rows
        mask = (df['Anunciante'].str.upper().str.contains('GOV')) & (df['Praça'].str.upper().str.contains('MARINGA')) & (df['Emissora TV'].str.upper().str.contains('RECORD'))
        
        # Count the number of rows that meet the filter criteria in your main DataFrame (df)
        num_filtered_rows_in_df = len(df[mask])
        
        if num_filtered_rows_in_df > 0:
            # Distribute the sum_value equally among the filtered rows in your main DataFrame
            value_per_row = sum_value / num_filtered_rows_in_df
            
            # Update the 'Valor Líquido Projetado' column for the filtered rows
            df.loc[mask, 'Valor Líquido Projetado'] = value_per_row

# Apply the function to each row
df.apply(set_govprices, args=(Bdf,), axis=1)

def set_assemprices(row, Bdf):
    anunciante = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    
    if 'ASSEMBLEIA' in anunciante and 'LONDRINA' in praca and 'RECORD' in emissora:
        filtered_rows = Bdf[(Bdf['Contato'] == 'LONDRINA') & (Bdf['Exibição'].str.contains('MARINGÁ'))]
        
        sum_value = filtered_rows['Valor Líquido'].sum()
    
        # Debug print statements
        print(f"Row Index: {row.name}")
        print(f"Filtered Rows:\n{filtered_rows}")
    
        # Create a mask for the filtered rows
        mask = (df['Anunciante'].str.upper().str.contains('ASSEMBLEIA')) & (df['Praça'].str.upper().str.contains('MARINGA')) & (df['Emissora TV'].str.upper().str.contains('RECORD'))
        
        # Count the number of rows that meet the filter criteria in your main DataFrame (df)
        num_filtered_rows_in_df = len(df[mask])
        
        if num_filtered_rows_in_df > 0:
            # Distribute the sum_value equally among the filtered rows in your main DataFrame
            value_per_row = sum_value / num_filtered_rows_in_df
            
            # Update the 'Valor Líquido Projetado' column for the filtered rows
            df.loc[mask, 'Valor Líquido Projetado'] = value_per_row

# Apply the function to each row

df.apply(set_assemprices, args=(Bdf,), axis=1)

# Print the modified DataFrame
print(df)

### Print our Gov and Assem updated prices

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Anunciante'].str.contains('GOV', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Anunciante'].str.contains('ASSEMBLEIA', na=False)

    dfprint = df[(condition1 | condition3) & condition2]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

# Curitiba Fix

In [ ]:
#so, now that we filled what we needed from the gov and assem, we'll start filling our clients, let's make our filter!
condition1 = Bdf['Emp. Venda'] == 23

condition2 = Bdf['Exibição'] == 'CURITIBA'

#now, that we've filtered everything, we'll start making some stuff
Bdf_CWB = Bdf[condition1]

#now, we group some data:
Bdf_CWB_group = Bdf_CWB.groupby(['Cliente']).agg({'Valor Líquido': 'sum'}).reset_index()

#print this shit out
print(Bdf_CWB_group)

In [ ]:
#now we know that some or most of our clients might have different names than what it's in our df 
client_mapping = {
    'SERALLE CALCADOS' : ['TERRITORIO DA AGUIA COMERCIO DE CALCADOS - EIRELI - EPP', 'CASTARDO COMERCIAL DE CALÇADOS LTDA', 'SERALLE COMERCIO DE CALCADOS LTDA'],
    'OTICA COMERCIAL' : 'ESCORPIAO JOIAS E RELOGIOS LTDA',
    'CASTRO E ROSA' : 'CASTRO E ROSA LTDA',
    'LOJAS MM MERCADOMOVEIS' : 'MERCADOMOVEIS LTDA',
    'ASSOC DOS LOJISTAS DO PALLADIUM SHOPPING CENTER' : 'ASSOCIAÇÃO DOS LOJISTAS DO PALLADIUM SHOPPING CENTER',
    'ASSOC PARA O FUNDO DE PROMOCAO DO VENTURA SHOPPING' : 'ASSOCIACAO PARA O FUNDO DE PROMOÇÃO DE VENTURA SHOPPING DE DESCONTOS',
    'BARIGUI VEICULOS' : 'AUTOBARIGUI COMERCIO DE VEICULOS LTDA',
    'TOKO FRIOS ALIMENTOS' : 'TOKO FRIOS ALIMENTOS LTDA.ME',
    'DAJU' : 'DAJU COMERCIO DE TECIDOS LTDA',
    'CONDOR SUPER CENTER' : 'CONDOR SUPER CENTER LTDA',
    'GIGANTE ATACADISTA' : 'GARANTE DISTRIB E IMPORT DE PROD ALIMENTICIOS LTDA',
    'MOVEIS CAMPO LARGO IND E COM' : 'MOVEIS CAMPO LARGO INDUSTRIA E COMERCIO  LTDA',
    'UNITECH IND DE MAQUINAS' : 'UNITECH INDUSTRIA DE MAQUINAS LTDA',
    'O SOLUCIONADOR ASSESSORIA' : ['O SOLUCIONADOR CURITIBA ASSESSORIA LTDA', 'O SOLU MARINGA ASSESSRIA FINANCEIRO LTDA', 'O SOLUCIONADOR LONDRINA ASSESSORIA LTDA'],
    'SUPER MUFFATO' : ['IRMAOS MUFFATO CIA LTDA', 'IRMAOS MUFFATO S.A'],
    'FARMACIAS NISSEI' : 'FARMÁCIAS E DROGARIAS NISSEI LTDA',
    'CIA BEAL DE ALIMENTOS' : 'CIA BEAL DE ALIMENTOS',
    'NATUCLIN' : 'NATUCLIN COMÉRCIO DE PRODUTOS NATURAIS E MANUFATURADOS LTDA',
    'AUTOSHOPPING LINHA VERDE' : 'ASSOCIACAO DOS LOJISTAS DO AUTO SHOPPING LINHA VERDE',
    'AUTO SHOPPING LINHA VERDE' : 'ASSOCIACAO DOS LOJISTAS DO AUTO SHOPPING LINHA VERDE',
    'SUPERMERCADO JACOMAR' : 'SUPERMERCADO JACOMAR LTDA',
    'INST ODONTOLOGICO DR CHRISTIAN ANDRADE' : 'C & R ODONTOLOGIA LTDA',
    'O BOTICARIO' : 'BOTICARIO PRODUTOS DE BELEZA LTDA',
    'FG EMPREENDIMENTOS' : 'FG BRAZIL HOLDING LTDA',
    'SERCOMTEL TELECOMUNICACOES' : 'LIGGA TELECOM',
    'BIRD EVENTOS' : 'MINASI & CIA LTDA',
    'UNICESUMAR' : 'CENTRO DE ENSINO SUPERIOR MARINGA LTDA - UNICESUMAR',
    'COAMO AGROINDUSTRIAL COOP' : 'COAMO COOPERATIVA AGROINDUSTRIAL',
    'BMG FOODS' : 'BMG FOODS IMPORTAÇÃO E EXPORTAÇÃO LTDA',
    'MOVEIS GAZIN' : 'GAZIN INDUSTRIA E COMERCIO DE MOVEIS E ELETRODOMESTICOS LTDA',
    'PREVER SERVICOS POSTUMOS' : 'PREVER SERVIÇOS POSTUMOS LTDA',
    'CIA SULAMERICANA DE DISTR' : 'COMPANHIA SULAMERICANA DE DISTRIBUICAO - CSD',
    'IGUARACU EMPRE IMOBILIARIOS' : 'IGUARACU EMPREENDIMENTOS IMOBILIARIOS LTDA - ME',
    'LOJA DA CASA' : 'M S G PISOS CERAMICOS LTDA',
    'ORAL TIME SERVICOS ODONTOLOGICOS' : 'ORAL TIME SERIÇOS ODONTOLOGICOS LTDA',
    'LOJAS ALVORADA' : 'LOJAS ALVORADA LTDA - ME',
    'IMPACTO FRANQUIAS' : 'J A L IMPACTO MARINGA COMERCIO DE PECAS E PNEUS EIRELI',
    'PONTO TRACK RASTREAMENTO E LOGISTICA' : 'PONTO TRACK RASTREAMENTO E LOGISTICA LTDA ME',
    'SATTRACK RASTREAMENTO E LOGISTICA' : 'SATTRACK RASTREAMENTO E LOGISTICA LTDA - ME',
    'SUPERMERCADO TONHAO' : 'SUPERMERCADO TONHAO LTDA ME',
    'VERONA SUPERMERCADOS' : 'SANCHES E VECCHIATE LTDA',
    'SERCOMTEL TELECOMUNICACOES' : 'SERCOMTEL S/A TELECOMUNICAÇÕES',
    'MOLICENTER' : 'MOLICENTER SUPERMERCADO LTDA',
    'SUPER PUPPO SUPERMERCADOS' : 'SUPER PUPPO SUPERMERCADOS LTDA',
    'SUPERMERCADO IRANI' : 'SUPERMERCADOS IRANI LTDA',
    'LOVAT HYUNDAI' : 'LOVAT VEICULOS S/A',
    'LOVAT HYUNDAI' : 'LOVAT VEÍCULOS LTDA',
    'SUPERDIA ATACADO' : ['ITALO SUPERMERCADO LTDA', 'ITALO SUPERMERCADOS LTDA'],
    'HOSPITAL DE OLHOS DE CASCAVEL' : 'HOSPITAL DE OLHOS CENTRO OFTALMOLOGICO DE CASCAVEL LTDA',
    'HIPERZOO PET SHOP' : 'HIPERZOO PET SHOP LTDA',
    'ALTHAIA' : 'ALTHAIA S.A. INDUSTRIA FARMACEUTICA',
    'ALIANCA MOVEIS' : 'COMERCIO DISTRIBUIDOR DE MOVEIS LTDA',
    'INST DOS OCULOS PR' : 'INSTITUTO DOS OCULOS FRANCHISING EIRELLI - ME',
    'SHOPPING DOS ENXOVAIS' : 'BMQ ENXOVAIS EIRELI',
    'FOLHA DE LONDRINA' : 'EDITORA E GRAFICA PARANA PRESS S.A.',
    'MOVEIS BRASILIA' : 'COMERCIAL DE MÓVEIS BRASÍLIA LTDA',
    'Y AGITA COM DE CALCADOS' : 'Y AGITA COMÉRCIO DE CALÇADOS LTDA',
    #PREFEITURAS:
    'PREF MUN CURITIBA (GMP)' : 'MUNICIPIO DE CURITIBA',
    'PREF MUN GUARATUBA (GMP)' : 'MUNICIPIO DE GUARATUBA',
    'PREF MUN FAZEN RIO GRANDE (GMP)' : 'PREFEITURA MUNICIPAL FAZENDA RIO GRANDE',
    'PREF MUN ARAUCARIA (GMP)' : 'PREFEITURA DO MUNICIPIO DE ARAUCARIA',
    'PREF MUN MARINGA (GMP)' : 'PREFEITURA DO MUNICIPIO DE MARINGA',
    'PREF MUN LONDRINA (GMP)' : 'PREFEITURA DO MUNICIPIO DE LONDRINA',
    'PREF MUN CASCAVEL (GMP)' : 'MUNICIPIO DE CASCAVEL',
    'PREF MUN TOLEDO (GMP)' : 'MUNICIPIO DE TOLEDO',
    'PREF MUN FOZ DO IGUACU (GMP)' : 'FOZ DO IGUACU PREFEITURA'
}

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'CURITIBA' in praca and emissora == 'RECORD TV' and coverage == 'CTBA':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('CURITIBA')) & (Bdf['Emp. Venda'] == 23)]  
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
            # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('CURITIBA')) & (df['Cobertura'].str.upper() == 'CTBA') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])

            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('CTBA', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('CURITIBA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

#### We'll fix Curitiba to other Cities

###### Curitiba to Londrina

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'LONDRINA' in praca and emissora == 'RECORD TV' and coverage == 'CTBA':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('LONDRINA')) & (Bdf['Emp. Venda'] == 23)]  
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
            additional_filters = (df['Praça'].str.contains('LONDRINA')) & (df['Cobertura'].str.upper() == 'CTBA') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('CTBA', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('LONDRINA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

###### Curitiba to Maringa

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'MARINGA' in praca and emissora == 'RECORD TV' and coverage == 'CTBA':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('MARINGÁ')) & (Bdf['Emp. Venda'] == 23)] 
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
            additional_filters = (df['Praça'].str.contains('MARINGA')) & (df['Cobertura'].str.upper() == 'CTBA') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('CTBA', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('MARINGA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

###### Curitiba to Cascavel

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'CASCAVEL' in praca and emissora == 'RECORD TV' and coverage == 'CTBA':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('TOLEDO')) & (Bdf['Emp. Venda'] == 23)] 
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
            additional_filters = (df['Praça'].str.contains('CASCAVEL')) & (df['Cobertura'].str.upper() == 'CTBA') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('CTBA', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('CASCAVEL', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

# Londrina Fix

In [ ]:
#so, now that we filled what we needed from the gov and assem, we'll start filling our clients, let's make our filter!
condition1 = Bdf['Emp. Venda'] == 25

condition2 = Bdf['Exibição'] == 'LONDRINA'

#now, that we've filtered everything, we'll start making some stuff
Bdf_LON = Bdf[condition1 & condition2]

#now, we group some data:
Bdf_LON_group = Bdf_LON.groupby(['Cliente']).agg({'Valor Líquido': 'sum'}).reset_index()

#print this shit out
print(Bdf_LON_group)

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'LONDRINA' in praca and emissora == 'RECORD TV' and coverage == 'LON':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('LONDRINA')) & (Bdf['Emp. Venda'] == 25)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('LONDRINA')) & (df['Cobertura'].str.upper() == 'LON') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('LON', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('LONDRINA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

#### We'll Fix Londrina to other Cities

###### Londrina to Curitiba

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'CURITIBA' in praca and emissora == 'RECORD TV' and coverage == 'LON':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('CURITIBA')) & (Bdf['Emp. Venda'] == 25)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
            # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('CURITIBA')) & (df['Cobertura'].str.upper() == 'LON') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('LON', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('CURITIBA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

###### Londrina to Maringa

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'MARINGA' in praca and emissora == 'RECORD TV' and coverage == 'LON':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('MARINGÁ')) & (Bdf['Emp. Venda'] == 25)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('MARINGA')) & (df['Cobertura'].str.upper() == 'LON') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('LON', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('MARINGA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

###### Londrina to Cascavel

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'CASCAVEL' in praca and emissora == 'RECORD TV' and coverage == 'LON':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('TOLEDO')) & (Bdf['Emp. Venda'] == 25)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('CASCAVEL')) & (df['Cobertura'].str.upper() == 'LON') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('LON', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('CASCAVEL', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

# Maringá Fix

In [ ]:
#so, now that we filled what we needed from the gov and assem, we'll start filling our clients, let's make our filter!
condition1 = Bdf['Emp. Venda'] == 24

condition2 = Bdf['Exibição'] == 'MARINGÁ'

#now, that we've filtered everything, we'll start making some stuff
Bdf_MAR = Bdf[condition1 & condition2]

#now, we group some data:
Bdf_MAR_group = Bdf_MAR.groupby(['Cliente']).agg({'Valor Líquido': 'sum'}).reset_index()

#print this shit out
print(Bdf_MAR_group)

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'MARINGA' in praca and emissora == 'RECORD TV' and coverage == 'MAR':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('MARINGÁ')) & (Bdf['Emp. Venda'] == 24)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('MARINGA')) & (df['Cobertura'].str.upper() == 'MAR') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('MAR', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('MARINGA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

#### We'll Fix Maringá to other Cities

##### Maringa to Curitiba

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'CURITIBA' in praca and emissora == 'RECORD TV' and coverage == 'MAR':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('CURITIBA')) & (Bdf['Emp. Venda'] == 24)]   
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('CURITIBA')) & (df['Cobertura'].str.upper() == 'MAR') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('MAR', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('CURITIBA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

##### Maringa to Cascavel

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'CASCAVEL' in praca and emissora == 'RECORD TV' and coverage == 'MAR':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('TOLEDO')) & (Bdf['Emp. Venda'] == 24)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('CASCAVEL')) & (df['Cobertura'].str.upper() == 'MAR') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('MAR', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('CASCAVEL', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

##### Maringa to Londrina

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'LONDRINA' in praca and emissora == 'RECORD TV' and coverage == 'MAR':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('LONDRINA')) & (Bdf['Emp. Venda'] == 24)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('LONDRINA')) & (df['Cobertura'].str.upper() == 'MAR') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('MAR', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('LONDRINA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

# Cascavel Fix

In [ ]:
#so, now that we filled what we needed from the gov and assem, we'll start filling our clients, let's make our filter!
condition1 = Bdf['Emp. Venda'] == 26

condition2 = Bdf['Exibição'] == 'TOLEDO'

#now, that we've filtered everything, we'll start making some stuff
Bdf_OESTE = Bdf[condition1 & condition2]

#now, we group some data:
Bdf_OESTE_group = Bdf_OESTE.groupby(['Cliente']).agg({'Valor Líquido': 'sum'}).reset_index()

#print this shit out
print(Bdf_OESTE_group)

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'CASCAVEL' in praca and emissora == 'RECORD TV' and coverage == 'OESTE':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('TOLEDO')) & (Bdf['Emp. Venda'] == 26)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('CASCAVEL')) & (df['Cobertura'].str.upper() == 'OESTE') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('OESTE', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('CASCAVEL', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

#### We'll Fix Cascavel to other Cities

##### Cascavel to Curitiba

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'CURITIBA' in praca and emissora == 'RECORD TV' and coverage == 'OESTE':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('CURITIBA')) & (Bdf['Emp. Venda'] == 26)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('CURITIBA')) & (df['Cobertura'].str.upper() == 'OESTE') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('OESTE', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('CURITIBA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

##### Cascavel to Londrina

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'LONDRINA' in praca and emissora == 'RECORD TV' and coverage == 'OESTE':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('LONDRINA')) & (Bdf['Emp. Venda'] == 26)] 
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('LONDRINA')) & (df['Cobertura'].str.upper() == 'OESTE') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('OESTE', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('LONDRINA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

##### Cascavel to Maringá

In [ ]:
# Define a custom function to update 'Valor Líquido Projetado'
def update_valor_liquido(row):
    client_name_df = row['Anunciante'].upper()
    praca = row['Praça'].upper()
    emissora = row['Emissora TV'].upper()
    coverage = row['Cobertura'].upper()
    
    # Check if the client name has a mapping
    if client_name_df in client_mapping:
        client_name_bdf = client_mapping[client_name_df]
    else:
        client_name_bdf = client_name_df
    
    # Apply additional filtering conditions
    if 'MARINGA' in praca and emissora == 'RECORD TV' and coverage == 'OESTE':
        # Check if the 'Valor Líquido Projetado' column is NaN or 0 before modifying it
        if pd.isna(row['Valor Líquido Projetado']) or row['Valor Líquido Projetado'] == 0:
            # Use client_name_bdf to filter Bdf and calculate the price
            filtered_rows = Bdf[(Bdf['Cliente'] == client_name_bdf) & (Bdf['Exibição'].str.contains('MARINGÁ')) & (Bdf['Emp. Venda'] == 26)]
            
            total_price = filtered_rows['Valor Líquido'].sum()  # Calculate the total price
            
             # Define your additional filter criteria
            additional_filters = (df['Praça'].str.contains('MARINGA')) & (df['Cobertura'].str.upper() == 'OESTE') & (df['Emissora TV'].str.upper() == 'RECORD TV')

            # Count the number of rows that meet the filter criteria in your main DataFrame (df)
            num_filtered_rows_in_df = len(df[(df['Anunciante'].str.upper() == client_name_df) & additional_filters])
            
            # Calculate the price per row
            price_per_row = total_price / num_filtered_rows_in_df
            
            return price_per_row
    return row['Valor Líquido Projetado']

# Use apply to update 'Valor Líquido Projetado' column
df['Valor Líquido Projetado'] = df.apply(update_valor_liquido, axis=1)
print(df)

In [ ]:
#made for Debug Purposes only, to test if the columns were correctly fulfilled
if 'Anunciante' in df.columns:
    condition1 = df['Cobertura'].str.contains('OESTE', na=False)
    condition2 = df['Emissora TV'].str.contains('RECORD', na=False)
    condition3 = df['Praça'].str.contains('MARINGA', na=False)
    condition4 = df['Desconto'] == 0.0
    condition5 = df['Valor Líquido Projetado'] != 0

    dfprint = df[condition1 & condition2 & condition3 & condition4 & condition5]

    print(dfprint)
else:
    print("Column 'Anunciante' does not exist in the DataFrame.")

# Let's make an excel file from this bad boy

In [ ]:
file_path = r'C:\Users\guilherme.oliveira\Desktop\programa_massa\archives\MonitorFlex_Julho.xlsx'

df.to_excel(file_path, index=False)